In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import cohen_kappa_score 


# combine all document items into list of lists
def mk_list(_items):
    # use only these columns for computations
    cols = ['al711233', 'al722382', 'al734634', 'al713696', 'al733357']
    items = []
    for c in cols:
        items.append(_items[c].values.tolist())
    return items  

# flatten list of lists (needed for computation of Cohen's kappa)
def flatten(list_in):
    flat_list = []
    for sublist in list_in:
        for item in sublist:
            flat_list.append(item)

    return flat_list

# Get Cohen's kappa and percent agreement
def irr(r1, r2):
    
    # verify row order between input files
    r1_ae = r1['annotation elements (34)']
    r2_ae = r2['annotation elements (34)']
    mm = pd.concat([r1_ae, r2_ae], axis=1)
    mm['mismatch'] = np.where((r1_ae != r2_ae), 1, 0)
    #print(mm[mm.mismatch == 1])
    if (mm[mm['mismatch'] != 1].empty):
        print('Mismatch on row annotations')
        exit 

    r1_items = mk_list(r1)
    r2_items = mk_list(r2)
    
    # verify document lengths
    if(len(flatten(r1_items)) == len(flatten(r2_items))):
        n_items = len(flatten(r2_items))

        # compute number of matched items
        n = 0
        for i in range(n_items):
            if (flatten(r1_items)[i] == flatten(r2_items)[i]):
                n += 1

        kappa = cohen_kappa_score(flatten(r1_items), flatten(r2_items))

        #print(f'Total matches across all documents: {n}')
        #print(f'Total number of annotated items: {n_items}')
        
        return ((n/n_items)*100, kappa)
    else:
        print('Invalid number of annotations!')

if __name__ == '__main__':
    
    # read in EL's annotations
    r1 = pd.read_excel('/Users/gms/Downloads/Trauma_IRR_EL_2.xlsx')
    r1 = r1[['annotation elements (34)', 'al711233', 'al722382', 'al734634', 'al713696', 'al733357']]
    # read in GR's annotations
    r2 = pd.read_excel('/Users/gms/Downloads/Trauma_IRR_GR_2.xlsx')
    r2 = r2[['annotation elements (34)', 'al711233', 'al722382', 'al734634', 'al713696', 'al733357']]     

    pa, k = irr(r1, r2)
    print(f'Percent agreement: {pa}')
    print(f'Cohen\'s kappa for all items combined: {k}')
    


Percent agreement: 93.33333333333333
Cohen's kappa for all items combined: 0.8659038083318433
